In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install pytorch_lightning==1.7.0 bert_score click keybert optuna torchmetrics==0.11.4 optuna==2.10.1 Levenshtein sacrebleu sacremoses summarizer

In [3]:
import torch
#torch.multiprocessing.set_start_method('forkserver', force=True)
from pathlib import Path
import sys

from drive.MyDrive.Group_24.preprocessor import WIKI_DOC, D_WIKI, EXP_DIR,MILDSUM
import time
import json

#from contextlib import contextmanager
import argparse

from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

# from T5_2 import SumSim, train
# from Bart2 import SumSim, train
from drive.MyDrive.Group_24.Bart_baseline_finetuned import BartBaseLineFineTuned, train
# from T5_baseline_finetuned import T5BaseLineFineTuned, train


def parse_arguments():
    p = ArgumentParser()

    p.add_argument('--seed', type=int, default=42, help='randomization seed')

    # p = SumSim.add_model_specific_args(p)
    p = BartBaseLineFineTuned.add_model_specific_args(p)
    # p = T5BaseLineFineTuned.add_model_specific_args(p)
    p = pl.Trainer.add_argparse_args(p)
    args,_ = p.parse_known_args()
    return args

# class MetricsCallback(pl.Callback):
#   def __init__(self):
#     super().__init__()
#     self.metrics = []

#   def on_validation_end(self, trainer, pl_module):
#       self.metrics.append(trainer.callback_metrics)

# Create experiment directory
def get_experiment_dir(create_dir=False):
    dir_name = f'{int(time.time() * 1000000)}'
    path = EXP_DIR / f'exp_{dir_name}'
    if create_dir == True: path.mkdir(parents=True, exist_ok=True)
    return path

def log_params(filepath, kwargs):
    filepath = Path(filepath)
    kwargs_str = dict()
    for key in kwargs:
        kwargs_str[key] = str(kwargs[key])
    json.dump(kwargs_str, filepath.open('w'), indent=4)



def run_training(args, dataset):

    args.output_dir = get_experiment_dir(create_dir=True)
    # logging the args
    log_params(args.output_dir / "params.json", vars(args))

    args.dataset = dataset
    print("Dataset: ",args.dataset)
    train(args)



dataset = MILDSUM
args = parse_arguments()
run_training(args, dataset)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Dataset:  MildSum
Initialize model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


MildSum
MildSum


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Sari score:  38.548429812120375
Val_loss 0.6145157018787962
Sari score:  38.47839055256048
Val_loss 0.6152160944743952
Sari score:  34.01206638735628
Val_loss 0.6598793361264372


INFO:pytorch_lightning.utilities.rank_zero:Epoch 6, global step 84: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=7` reached.


Sari score:  36.173454949473644
Val_loss 0.6382654505052636
train_loss tensor(2.2674, device='cuda:0')
val_loss tensor(0.6320, device='cuda:0')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


training finished
Saving model


In [2]:
import click
from drive.MyDrive.Group_24.easse.utils.resources import get_orig_sents, get_refs_sents

def get_sys_sents(test_set, sys_sents_path=None):
    # Get system sentences to be evaluated
    if sys_sents_path is not None:
        return read_lines(sys_sents_path)
    else:
        # read the system output
        with click.get_text_stream("stdin", encoding="utf-8") as system_output_file:
            return system_output_file.read().splitlines()


def get_orig_and_refs_sents(test_set, orig_sents_path=None, refs_sents_paths=None):
    # Get original and reference sentences
    if test_set == "custom":
        assert orig_sents_path is not None
        assert refs_sents_paths is not None


        orig_sents = read_lines(orig_sents_path)
        # NOTE: refs_sents_paths is a list of paths
        refs_sents = [read_lines(refs_sents_paths)]
        # if type(refs_sents_paths) == str:
        #     refs_sents_paths = refs_sents_paths.split(",")
        #refs_sents = [read_lines(ref_sents_path) for ref_sents_path in refs_sents_paths]
    else:
        orig_sents = get_orig_sents(test_set)
        refs_sents = get_refs_sents(test_set)
    # Final checks
    # assert all(
    #     [len(orig_sents) == len(ref_sents) for ref_sents in refs_sents]
    # ), f'Not same number of lines for test_set={test_set}, orig_sents_path={orig_sents_path}, refs_sents_paths={refs_sents_paths}'  # noqa: E501
    return orig_sents, refs_sents

In [3]:
! pip install requests>=2.21.0 seaborn stanfordnlp yattag plotly>=4.0.0 tseval@git+https://github.com/facebookresearch/text-simplification-evaluation.git@main simalign

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-j1omkt1f/tseval_c86c134b4f194053a87b380ea2169e42


In [4]:
from pathlib import Path
import sys
import os
import shutil
from drive.MyDrive.Group_24.preprocessor import WIKI_DOC, D_WIKI, EXP_DIR,MILDSUM
# sys.path.append(str(Path(__file__).resolve().parent.parent))
#sys.path.append(str(Path(__file__).resolve().parent.parent))
# -- end fix path --

from drive.MyDrive.Group_24.easse.cli import evaluate_system_output
#from Ts_T5 import T5FineTuner
from drive.MyDrive.Group_24.easse.report import get_all_scores
from contextlib import contextmanager
import json
#from preprocessor import Preprocessor
import torch
#from transformers import T5ForConditionalGeneration, T5TokenizerFast
from drive.MyDrive.Group_24.preprocessor import get_data_filepath, EXP_DIR,  REPO_DIR, WIKI_DOC, D_WIKI
from drive.MyDrive.Group_24.preprocessor import write_lines, yield_lines, count_line, read_lines, generate_hash
from drive.MyDrive.Group_24.easse.sari import corpus_sari
import time
from drive.MyDrive.Group_24.utils.D_SARI import D_SARIsent, D_SARI_file
#from googletrans import Translator
#from Bart2 import SumSim
# from T5_2 import SumSim
# from T5_baseline_finetuned import T5BaseLineFineTuned
from drive.MyDrive.Group_24.Bart_baseline_finetuned import BartBaseLineFineTuned


@contextmanager
def log_stdout(filepath, mute_stdout=False):
    '''Context manager to write both to stdout and to a file'''

    class MultipleStreamsWriter:
        def __init__(self, streams):
            self.streams = streams

        def write(self, message):
            for stream in self.streams:
                stream.write(message)

        def flush(self):
            for stream in self.streams:
                stream.flush()

    save_stdout = sys.stdout
    log_file = open(filepath, 'w')
    if mute_stdout:
        sys.stdout = MultipleStreamsWriter([log_file])  # Write to file only
    else:
        sys.stdout = MultipleStreamsWriter([save_stdout, log_file])  # Write to both stdout and file
    try:
        yield
    finally:
        sys.stdout = save_stdout
        log_file.close()

# set random seed universal
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(42)
model_dir = None
_model_dirname = None
max_len = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

# specify the model_name and checkpoint_name

# model_dirname = 'exp_WikiDocFinal_T5_kw_num4_div0.9_0.001CosSim(ReLU_W)'
# checkpoint_path = 'checkpoint-epoch=5.ckpt'

#### Single model ####
# load the model
Model = BartBaseLineFineTuned.load_from_checkpoint('/content/drive/MyDrive/Group_24/experiments/exp_1714728438675372/checkpoint-epoch=5.ckpt').to(device)
model = Model.model.to(device)
tokenizer = Model.tokenizer
#### Single model ####

#### Joint model ####
# Model = SumSim.load_from_checkpoint('/content/experiments/checkpoint-epoch=2.ckpt').to(device)
# summarizer = Model.summarizer.to(device)
# simplifier = Model.simplifier.to(device)
# summarizer_tokenizer = Model.summarizer_tokenizer
# simplifier_tokenizer = Model.simplifier_tokenizer
#### Joint model ####

def generate_single(sentence, preprocessor = None):
    '''
    This function is for T5 or Bart single model to generate/predict
    '''

    # text = "simplify: " + sentence  ### -> for T5
    text = sentence
    encoding = tokenizer(text, max_length=512,
                                     padding='max_length',
                                     truncation=True,
                                     return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_masks = encoding["attention_mask"].to(device)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        num_beams=2,
        top_k=70,
        top_p=0.95,
        early_stopping=True,
        num_return_sequences=1,
    )
    sent = tokenizer.decode(beam_outputs[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return sent


# def generate(sentence, preprocessor=None):
#     '''
#     This function is for Joint model to generate/predict
#     '''

#     # For T5
#     #sentence = 'summarize: ' + sentence

#     encoding = summarizer_tokenizer(
#         [sentence],
#         max_length = 512,
#         truncation = True,
#         padding = 'max_length',
#         return_tensors = 'pt',
#     )

#     summary_ids = summarizer.generate(
#         encoding['input_ids'].to(device),
#         num_beams = 15,
#         min_length = 30,
#         max_length = 512,
#         top_k = 80, top_p = 0.97
#     ).to(device)

#     # For T5
#     # for i, summary_id in enumerate(summary_ids):
#     #     add_tokens = torch.tensor([18356, 10]).to(device)
#     #     summary_ids[i,:] = torch.cat((summary_id, add_tokens), dim=0)[:-2]

#     summary_atten_mask = torch.ones(summary_ids.shape).to(device)
#     summary_atten_mask[summary_ids[:,:] == summarizer_tokenizer.pad_token_id] = 0

#     beam_outputs = simplifier.generate(
#         input_ids = summary_ids,
#         attention_mask = summary_atten_mask,
#         do_sample = True,
#         max_length = 256,
#         num_beams = 5, #16
#         top_k = 80,  #120
#         top_p = 0.95, #0.95
#         early_stopping = True,
#         num_return_sequences = 1,
#     )

#     sent = simplifier_tokenizer.decode(beam_outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     return sent

    '''
    sentence = preprocessor.encode_sentence(sentence)
    text = "simplify: " + sentence
    encoding = tokenizer(text, max_length=max_len,
                                     padding='max_length',
                                     truncation=True,
                                     return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_masks = encoding["attention_mask"].to(device)

    # set top_k = 130 and set top_p = 0.97 and num_return_sequences = 1
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        do_sample=False,
        max_length=max_len,
        num_beams=10,
        top_k=130,
        top_p=0.97,
        early_stopping=True,
        num_return_sequences=1
    )
    pred_sent = tokenizer.decode(beam_outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return pred_sent
    # final_outputs = []
    # for beam_output in beam_outputs:
    #     sent = tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    #     if sent.lower() != sentence.lower() and sent not in final_outputs:
    #         final_outputs.append(sent)

    # return final_outputs
    '''


def evaluate(orig_filepath, sys_filepath, ref_filepaths):
    orig_sents = read_lines(orig_filepath)
    # NOTE: change the refs_sents if several references are used
    refs_sents = [read_lines(ref_filepaths)]
    #refs_sents = [read_lines(filepath) for filepath in ref_filepaths]

    return corpus_sari(orig_sents, read_lines(sys_filepath), refs_sents)

# def evaluate_on(dataset, features_kwargs, phase, model_dirname=None, checkpoint_path=None):

#     global model, tokenizer, device, model_dir, _model_dirname, max_len
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model_path = EXP_DIR / model_dirname / checkpoint_path
#     Model = T5FineTuner.load_from_checkpoint(model_path).to(device)
#     model = Model.model
#     tokenizer = Model.tokenizer
#     model_dir = EXP_DIR / model_dirname

#     # load_model(model_dirname)
#     preprocessor = Preprocessor(features_kwargs)
#     output_dir = model_dir / "outputs"
#     # output_dir = REPO_DIR / f"outputs/{_model_dirname}"
#     output_dir.mkdir(parents=True, exist_ok=True)
#     print("Output dir: ", output_dir)
#     features_hash = generate_hash(features_kwargs)
#     output_score_filepath = output_dir / f"score_{features_hash}_{dataset}_{phase}_log.txt"
#     if not output_score_filepath.exists() or count_line(output_score_filepath) == 0:
#         # log_params(output_dir / f"{features_hash}_features_kwargs.json", features_kwargs)
#         start_time = time.time()
#         complex_filepath = get_data_filepath(dataset, phase, 'complex')
#         pred_filepath = output_dir / f'{features_hash}_{complex_filepath.stem}.txt'
#         # ref_filepaths = [get_data_filepath(dataset, phase, 'simple.turk', i) for i in range(8)]
#         ref_filepath = get_data_filepath(dataset, phase, 'simple')
#         print(pred_filepath)
#         if pred_filepath.exists() and count_line(pred_filepath) == count_line(complex_filepath):
#             print("File is already processed.")
#         else:
#             simplify_file(complex_filepath, pred_filepath, preprocessor)

#         # print("Evaluate: ", pred_filepath)
#         with log_stdout(output_score_filepath):
#             # print("features_kwargs: ", features_kwargs)

#             # refs = [[line] for line in yield_lines(ref_filepath)]
#             # score = corpus_sari(read_lines(complex_filepath), read_lines(pred_filepath), refs)
#             # print(len(read_lines(complex_filepath)), len(read_lines(pred_filepath)) )
#             # print([len(s) for s in refs])

#             # scores = get_all_scores(read_lines(complex_filepath), read_lines(pred_filepath), refs)
#             score = evaluate(complex_filepath, pred_filepath, [ref_filepath])
#             # scores = evaluate_all_metrics(complex_filepath, pred_filepath, ref_filepaths)
#             if "WordRatioFeature" in features_kwargs:
#                 print("W:", features_kwargs["WordRatioFeature"]["target_ratio"], "\t", end="")
#             if "CharRatioFeature" in features_kwargs:
#                 print("C:", features_kwargs["CharRatioFeature"]["target_ratio"], "\t", end="")
#             if "LevenshteinRatioFeature" in features_kwargs:
#                 print("L:", features_kwargs["LevenshteinRatioFeature"]["target_ratio"], "\t", end="")
#             if "WordRankRatioFeature" in features_kwargs:
#                 print("WR:", features_kwargs["WordRankRatioFeature"]["target_ratio"], "\t", end="")
#             if "DependencyTreeDepthRatioFeature" in features_kwargs:
#                 print("DTD:", features_kwargs["DependencyTreeDepthRatioFeature"]["target_ratio"], "\t", end="")
#             print("{:.2f} \t ".format(score))

#             print("Execution time: --- %s seconds ---" % (time.time() - start_time))
#     else:
#         print("Already exist: ", output_score_filepath)
#         print("".join(read_lines(output_score_filepath)))

def back_translation(text):
    X = translator.translate(text, dest = 'de')
    return translator.translate(X.text, dest = 'en').text


def simplify_file(complex_filepath, output_filepath, features_kwargs=None, model_dirname=None, post_processing=True):
    '''
    Obtain the simplified sentences (predictions) from the original complex sentences.
    '''

    total_lines = count_line(complex_filepath)
    print(complex_filepath)
    print(complex_filepath.stem)

    output_file = Path(output_filepath).open("w")

    for n_line, complex_sent in enumerate(yield_lines(complex_filepath), start=1):
        ### NOTE: Change it when using Single model or Joint model
        output_sents = generate_single(complex_sent, preprocessor = None)
        # output_sents = generate(complex_sent, preprocessor=None)


        print(f"{n_line}/{total_lines}", " : ", output_sents)
        if output_sents:
            # output_file.write(output_sents[0] + "\n")
            output_file.write(output_sents + "\n")
        else:
            output_file.write("\n")
    output_file.close()

    if post_processing: post_process(output_filepath)

def post_process(filepath):
    lines = []
    for line in yield_lines(filepath):
        lines.append(line.replace("''", '"'))
    write_lines(lines, filepath)

def evaluate_on_MILDSUM(phase, features_kwargs=None,  model_dirname = None):
    dataset = MILDSUM
    #model_dir = EXP_DIR / model_dirname
    output_dir = '/content/outputs'

    #output_dir.mkdir(parents = True, exist_ok = True)

    if not os.path.exists(output_dir):
      os.makedirs(output_dir)

    #features_hash = generate_hash(features_kwargs)
    output_score_filepath = f'{output_dir}/score_{dataset}_{phase}.log.txt'
    #complex_filepath = get_data_filepath(dataset, phase, 'complex_kw_num4_div0.9') # _kw_num3_div0.9'
    complex_filepath = get_data_filepath(dataset, phase, 'complex')

    if not os.path.exists(output_score_filepath) or count_line(output_score_filepath)==0:
        start_time = time.time()
        #complex_filepath =get_data_filepath(dataset, phase, 'complex_kw_num4_div0.9')
        complex_filepath =get_data_filepath(dataset, phase, 'complex')
        complex_filepath = Path(complex_filepath)

        #complex_filepath = get_data_filepath(dataset, phase, 'complex_summary_'+str(ratio))
        pred_filepath = f'{output_dir}/{complex_filepath.stem}.txt'
        ref_filepaths = get_data_filepath(dataset, phase, 'simple')

        if os.path.exists(pred_filepath) and count_line(pred_filepath)==count_line(complex_filepath):
            print("File is already processed.")
        else:
            simplify_file(complex_filepath, pred_filepath, features_kwargs, model_dirname)

        print("Evaluate: ", pred_filepath)

        with log_stdout(output_score_filepath):

            scores  = evaluate_system_output(test_set='custom',
                                             sys_sents_path=str(pred_filepath),
                                             orig_sents_path=str(complex_filepath),
                                             refs_sents_paths=str(ref_filepaths),metrics = ['bleu', 'sari', 'fkgl'] )

            sys_sents = get_sys_sents(test_set = 'custom', sys_sents_path=str(pred_filepath))
            orig_sents, refs_sents = get_orig_and_refs_sents(test_set = 'custom',
                                                             orig_sents_path = str(complex_filepath),
                                                             refs_sents_paths = str(ref_filepaths))


            print("SARI: {:.2f}\t D-SARI: {:.2f} \t BLEU: {:.2f} \t FKGL: {:.2f} ".format(scores['sari'],
                                                                                          D_SARI_file(orig_sents,sys_sents,refs_sents[0],),
                                                                                          scores['bleu'],
                                                                                          scores['fkgl']))
            # print("{:.2f} \t {:.2f} \t {:.2f} ".format(scores['SARI'], scores['BLEU'], scores['FKGL']))

            print("Execution time: --- %s seconds ---" % (time.time() - start_time))
            return None
    else:
        print("Already exists: ", output_score_filepath)
        print("".join(read_lines(output_score_filepath)))

evaluate_on_MILDSUM(phase='test', features_kwargs=None, model_dirname=None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Already exists:  /content/outputs/score_MildSum_test.log.txt
SARI: 38.00	 D-SARI: 30.88 	 BLEU: 0.79 	 FKGL: 7.18Execution time: --- 1.9902255535125732 seconds ---


In [16]:
# Path to the zip file
zip_file_path = '/content/outputs.zip'

if os.path.exists(zip_file_path):
    os.remove(zip_file_path)

#Compress the folder into a zip file
!zip -r {zip_file_path} {os.path.basename('/content/outputs')}

  adding: outputs/ (stored 0%)
  adding: outputs/score_MildSum_test.log.txt (deflated 7%)
  adding: outputs/MildSum.test.txt (deflated 65%)


In [17]:
from google.colab import files
files.download(f'/content/outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>